# Projeto 1 - Ciência dos Dados

Nome: Carlos Eduardo Porciuncula Yamada

Nome: Pedro Henrique de Sousa da Silva

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

* Tweets relevantes:
    - Críticas à marca ou produtos
    - Anúncios de eventos
    - Venda de produtos


___
Carregando algumas bibliotecas:

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

In [2]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
c:\Users\Paula Yamada\OneDrive\Área de Trabalho\2021.2\cdados\cdados-projeto-1


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [3]:
filename = 'Apple.xlsx'

In [4]:
train = pd.read_excel(filename)
train.head(5)

,Treinamento,Classificação
0,e já temos uma hashflag ativada para o evento ...,1
1,o amo tanto https://t.co/8q8kwbchxg,1
2,apple ta a oferecer airpods na compra de um ma...,1
3,"primeiro eu e a larissa compra o airpods pro, ...",1
4,@autcoliveira eu tô na minha cabeça você é a a...,0


In [5]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

,Teste,Classificação
0,apple music horrível dou play e a música não t...,1
1,brilha brilha https://t.co/qdluo54yjn,0
2,muitos sonham em ter essa na discografia mas s...,0
3,vocals https://t.co/knkwca4vwn,0
4,#apple #iphone - bloquez vos agendas 🧐😎 https:...,1


___
## Classificador automático de sentimento


Apple é uma empresa norte-americana que recentemente anunciou o lançamento de diversos produtos como o iPhone 13 e o Apple Watch 7.

A classificação foi feita considerando se o tweet tem relação com a marca ou se faz parte de uma discussão relacionada à empresa: opinião, crítica ou afim.

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [47]:
train['Treinamento'] = train['Treinamento'].astype('category')
test['Teste'] = test['Teste'].astype('category')

In [49]:
import re 

def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    punctuation = '[!-.:?;]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    return text_subbed

In [50]:
lista = list()
for i in range(train.shape[0]):
    lista += cleanup(train.Treinamento[i]).split()
serie_twt = pd.Series(lista)
twt_relat = serie_twt.value_counts(True)

In [51]:
bool_ = train.isin([1])

filtro_r = bool_['Classificação'] == True
filtro_i = bool_['Classificação'] == False

relevantes = train.loc[filtro_r, :]
irrelevantes = train.loc[filtro_i, :]

In [52]:
li_rel = list()

for i in range(relevantes.shape[0]):
    li_rel += cleanup(relevantes.Treinamento[relevantes.index[i]]).split()

serie_rel = pd.Series(li_rel)
rel_relat = serie_rel.value_counts(True)

In [53]:
li_irr = list()

for i in range(irrelevantes.shape[0]):
    li_irr += cleanup(irrelevantes.Treinamento[irrelevantes.index[i]]).split()

serie_irr = pd.Series(li_irr)
irr_relat = serie_irr.value_counts(True)

In [54]:
probR = len(li_rel)/len(lista)

probI = len(li_irr)/len(lista)

In [55]:
serie = pd.Series(cleanup(test.Teste[0]).split())

* $P(R|tweet) = \frac{P(tweet|R)P(R)}{P(tweet)}$

* $P(tweet|R) = P(p_1|R)*P(p_2|R)*...*P(p_n|R)$

In [140]:
test['Naive'] = ''

In [141]:
test

,Teste,Classificação,Naive
0,apple music horrível dou play e a música não t...,1,
1,brilha brilha https://t.co/qdluo54yjn,0,
2,muitos sonham em ter essa na discografia mas s...,0,
3,vocals https://t.co/knkwca4vwn,0,
4,#apple #iphone - bloquez vos agendas 🧐😎 https:...,1,
...,...,...,...
195,"esse apple pay é um adianto, não preciso mais ...",1,
196,2:25 meu deus https://t.co/ybl9vck3j9,0,
197,a apple mudou o endereço da entrega por algum ...,1,
198,@dudaslz_ @vmoronicius amiga para a menina lá ...,1,


In [144]:
test

,Teste,Classificação,Naive
0,apple music horrível dou play e a música não t...,1,1
1,brilha brilha https://t.co/qdluo54yjn,0,0
2,muitos sonham em ter essa na discografia mas s...,0,1
3,vocals https://t.co/knkwca4vwn,0,0
4,#apple #iphone - bloquez vos agendas 🧐😎 https:...,1,1
...,...,...,...
195,"esse apple pay é um adianto, não preciso mais ...",1,0
196,2:25 meu deus https://t.co/ybl9vck3j9,0,1
197,a apple mudou o endereço da entrega por algum ...,1,1
198,@dudaslz_ @vmoronicius amiga para a menina lá ...,1,1


___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [147]:
for n in range(test.shape[0]):
    probTdadoI = 1
    probTdadoR = 1

    tweet = cleanup(test.Teste[n]).split()

    for p in tweet:
        if p in rel_relat:
            probTdadoR *= rel_relat[p]
    
    for p in tweet:
        if p in irr_relat:
            probTdadoI *= irr_relat[p]
    
    if probTdadoR > probTdadoI:
        test.Naive[n] = 1
    else:
        test.Naive[n] = 0

<ipython-input-147-83375ce90b27>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test.Naive[n] = 1
<ipython-input-147-83375ce90b27>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test.Naive[n] = 0


In [166]:
a = 0
for i in range(test.shape[0]):
    if test.Classificação[i] == test.Naive[i]:
        a += 1
print(f'- Exatidão: {a/test.shape[0]*100}%')

- Exatidão: 41.5%


___
### Concluindo

___
### Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nos tweets. Ex: stemming, lemmatization, stopwords
* CORRIGIU separação de espaços entre palavras e emojis ou entre emojis e emojis
* CRIOU categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante. Pelo menos quatro categorias, com adição de mais tweets na base, conforme enunciado. (OBRIGATÓRIO PARA TRIOS, sem contar como item avançado)
* EXPLICOU porquê não pode usar o próprio classificador para gerar mais amostras de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto
* SUGERIU e EXPLICOU melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* FEZ o item 6. Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste descrito no enunciado do projeto (OBRIGATÓRIO para conceitos A ou A+)

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**